#  Segmenting and Clustering Neighborhoods in Toronto (Part 2)

This notebook is part of Applied Data Science Capstone course @Coursera.

Postal data was scraped, engineered and saved to CSV in part 1. Here we simply load the CSV back in.

In [15]:
import pandas as pd
df = pd.read_csv('Canada_Postal.csv', index_col=0)
print(df.head())
print(df.shape)

  PostalCode           Borough                                 Neighborhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
(103, 3)


I had given up attempt to use geocoder as I was getting None value every single time. Downloading provided CSV file with Lat/Long data instead.

In [33]:
import wget
wget.download('http://cocl.us/Geospatial_data', 'coords.csv')

'coords.csv'

In [40]:
coords = pd.read_csv('coords.csv')
coords.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [47]:
df2 = pd.merge(df, coords, left_on=['PostalCode'],right_on=['Postal Code'],how='left')
df2.drop('Postal Code', axis=1, inplace=True)
df2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
